## Homework 2

In [ ]:
# Import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

We first find the appropriate lists giving us the needed data

In [ ]:
#base url for websites to crawl
TOP_UNIVERSITIES = 'https://www.topuniversities.com'
TIMES_EDUCATION = 'https://www.timeshighereducation.com'

#this is our main url
TIMES_EDUCATION_JSON = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'

TOP_UNIVERVERSITIES_TEXT = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt'

In [ ]:
#sending request and getting json
r = requests.get(TIMES_EDUCATION_JSON)
timesjson = r.json()

In [ ]:
#transforming into Data frame
times_frame = pd.DataFrame(timesjson['data']).head(200) #we only want first 200
#getting rid of useless data
times_frame = times_frame[[ d for d in times_frame.columns if not ('score' in d)]] # drop useless frames
del times_frame['member_level'], times_frame['nid'], times_frame['record_type'], times_frame['subjects_offered'], times_frame['rank']
#transforming data
times_frame['rank_order'] = times_frame['rank_order'].map(lambda x: int(int(x)/10)) #use absolute order as seen on website

times_frame.head()

In [ ]:
s = requests.get(TOP_UNIVERVERSITIES_TEXT)
topjson = s.json()

In [ ]:
top_frame = pd.DataFrame(topjson['data']).head(200) #we only want first 200
del top_frame['cc'], top_frame['logo'], top_frame['nid'], top_frame['core_id'], top_frame['stars'], top_frame['guide'] 

top_frame.head()

We now need to ensure name, rank, country and region, number of faculty members (international and total) and number of students (international and total) can be found in the df

In [ ]:
#example usage
r = requests.get(TOP_UNIVERSITIES+ '/universities/university-cambridge')
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
student_number = (soup.find('div', class_='total student')).find('div', class_='number').text[1:-1]
int_student = (soup.find('div', class_='total inter')).find('div', class_='number').text[1:-1]
facult_number = (soup.find('div', class_='total faculty')).find('div', class_='number').text[1:-1]
int_faculty = (soup.find('div', class_='inter faculty')).find('div', class_='number').text[1:-1]
pd.DataFrame.from_dict({'students' : [student_number], 
              'international students': [int_student], 
              'faculty' : [facult_number], 
              'international faculty' : [int_faculty] })

In [ ]:
def get_additional_info(url):
    r = requests.get(TOP_UNIVERSITIES+ url)
    page_body = r.text
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    student_number = (soup.find('div', class_='total student')) 
    if student_number:
        student_number = student_number.find('div', class_='number').text[1:]
        
    int_student = (soup.find('div', class_='total inter'))
    if int_student:
        int_student = int_student.find('div', class_='number').text[1:]    
    
    faculty_number = (soup.find('div', class_='total faculty'))
    if faculty_number:
        faculty_number = faculty_number.find('div', class_='number').text[1:]
        
    int_faculty = (soup.find('div', class_='inter faculty'))
    if int_faculty:
        int_faculty = int_faculty.find('div', class_='number').text[1:]
    
    frame = pd.DataFrame.from_dict({'students' : [student_number], 
              'international students': [int_student], 
              'faculty' : [faculty_number], 
              'international faculty' : [int_faculty] })
    return frame

In [ ]:
#takes very long to run, to explore data add field :)
missing_rows = pd.DataFrame()
for url in top_frame.url:
    new_info = get_additional_info(url)
    missing_rows = missing_rows.append(new_info)

missing_rows

In [ ]:
##Todo merge missing rows with topuniversities frame